# 25 epoch

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Subset
from medmnist import PathMNIST
import numpy as np

class SimCLR(nn.Module):
    def __init__(self, base_model, out_dim):
        super(SimCLR, self).__init__()
        self.encoder = self.get_resnet(base_model)
        self.projector = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, out_dim)
        )

    def get_resnet(self, base_model):
        model = models.__dict__[base_model](weights=None)
        model = nn.Sequential(*list(model.children())[:-1])
        return model

    def forward(self, x):
        h = self.encoder(x)
        h = h.squeeze()
        z = self.projector(h)
        return h, z


model = SimCLR(base_model='resnet18', out_dim=128).cuda()


try:
    model.load_state_dict(torch.load('./simclr_pathmnist_pretrained_epoch_25.pth'))
except Exception as e:
    print(f"Error loading model: {e}")


for param in model.encoder.parameters():
    param.requires_grad = False


class LinearProbe(nn.Module):
    def __init__(self, base_encoder):
        super(LinearProbe, self).__init__()
        self.encoder = base_encoder
        self.classifier = nn.Linear(512, 9)  # PathMNIST 有 9 个类别

    def forward(self, x):
        with torch.no_grad():
            h = self.encoder(x)
        h = h.squeeze()
        logits = self.classifier(h)
        return logits


linear_probe_model = LinearProbe(model.encoder).cuda()

PathMNIST_MEAN = [0.73765225, 0.53090023, 0.70307171]
PathMNIST_STD = [0.12319908, 0.17607205, 0.12394462]


data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=PathMNIST_MEAN, std=PathMNIST_STD)
])

train_dataset = PathMNIST(split='train', transform=data_transform, download=True, size=64)
val_dataset = PathMNIST(split='val', transform=data_transform, download=True, size=64)
test_dataset = PathMNIST(split='test', transform=data_transform, download=True, size=64)

num_samples = len(train_dataset)
subset_indices = list(range(num_samples))
np.random.shuffle(subset_indices)
subset_indices = subset_indices[:int(0.01 * num_samples)]

train_subset = Subset(train_dataset, subset_indices)
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(linear_probe_model.classifier.parameters(), lr=1e-3, weight_decay=1e-6)

def train_linear_probe(model, train_loader, criterion, optimizer, epoch):
    model.train()
    total_loss = 0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.cuda(), labels.squeeze().long().cuda() 
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        if batch_idx % 10 == 0:
            print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}')
    return total_loss / len(train_loader)

def evaluate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.cuda(), labels.squeeze().long().cuda()  # 确保 labels 是 1D 张量
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

num_epochs = 10
best_val_accuracy = 0
best_model_state = None
for epoch in range(num_epochs):
    train_loss = train_linear_probe(linear_probe_model, train_loader, criterion, optimizer, epoch)
    val_accuracy = evaluate(linear_probe_model, val_loader)
    print(f'Epoch {epoch}, Train Loss: {train_loss}, Validation Accuracy: {val_accuracy}%')
    
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model_state = linear_probe_model.state_dict()

linear_probe_model.load_state_dict(best_model_state)

test_accuracy = evaluate(linear_probe_model, test_loader)
print(f'Best Validation Accuracy: {best_val_accuracy}%')
print(f'Test Accuracy: {test_accuracy}%')


Error loading model: PytorchStreamReader failed reading zip archive: failed finding central directory
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Epoch 0, Batch 0, Loss: 2.318887233734131
Epoch 0, Batch 10, Loss: 2.0775582790374756
Epoch 0, Train Loss: 2.17932923634847, Validation Accuracy: 14.8140743702519%
Epoch 1, Batch 0, Loss: 2.0209834575653076
Epoch 1, Batch 10, Loss: 1.9749492406845093
Epoch 1, Train Loss: 2.0538632313410443, Validation Accuracy: 23.970411835265892%
Epoch 2, Batch 0, Loss: 2.0560190677642822
Epoch 2, Batch 10, Loss: 1.8476451635360718
Epoch 2, Train Loss: 1.9483352820078532, Validation Accuracy: 33.496601359456214%
Epoch 3, Batch 0, Loss: 1.9819947481155396
Epoch 3, Batch 10, Loss: 1.8649293184280396
Epoch 3, Train Loss: 1.9310323079427083, Validation Accuracy: 35.715713714514195%

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Subset
from medmnist import PathMNIST
import numpy as np

class SimCLR(nn.Module):
    def __init__(self, base_model, out_dim):
        super(SimCLR, self).__init__()
        self.encoder = self.get_resnet(base_model)
        self.projector = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, out_dim)
        )

    def get_resnet(self, base_model):
        model = models.__dict__[base_model](weights=None)
        model = nn.Sequential(*list(model.children())[:-1])
        return model

    def forward(self, x):
        h = self.encoder(x)
        h = h.squeeze()
        z = self.projector(h)
        return h, z

model = SimCLR(base_model='resnet18', out_dim=128).cuda()

try:
    model.load_state_dict(torch.load('./simclr_pathmnist_pretrained_epoch_25.pth'))
except Exception as e:
    print(f"Error loading model: {e}")

for param in model.encoder.parameters():
    param.requires_grad = False

class LinearProbe(nn.Module):
    def __init__(self, base_encoder):
        super(LinearProbe, self).__init__()
        self.encoder = base_encoder
        self.classifier = nn.Linear(512, 9)  # PathMNIST 有 9 个类别

    def forward(self, x):
        with torch.no_grad():
            h = self.encoder(x)
        h = h.squeeze()
        logits = self.classifier(h)
        return logits


linear_probe_model = LinearProbe(model.encoder).cuda()

PathMNIST_MEAN = [0.73765225, 0.53090023, 0.70307171]
PathMNIST_STD = [0.12319908, 0.17607205, 0.12394462]

data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=PathMNIST_MEAN, std=PathMNIST_STD)
])

train_dataset = PathMNIST(split='train', transform=data_transform, download=True, size=64)
val_dataset = PathMNIST(split='val', transform=data_transform, download=True, size=64)
test_dataset = PathMNIST(split='test', transform=data_transform, download=True, size=64)

num_samples = len(train_dataset)
subset_indices = list(range(num_samples))
np.random.shuffle(subset_indices)
subset_indices = subset_indices[:int(0.1 * num_samples)]

train_subset = Subset(train_dataset, subset_indices)
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(linear_probe_model.classifier.parameters(), lr=1e-3, weight_decay=1e-6)

def train_linear_probe(model, train_loader, criterion, optimizer, epoch):
    model.train()
    total_loss = 0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.cuda(), labels.squeeze().long().cuda()  # 确保 labels 是 1D 张量
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        if batch_idx % 10 == 0:
            print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}')
    return total_loss / len(train_loader)

def evaluate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.cuda(), labels.squeeze().long().cuda()  # 确保 labels 是 1D 张量
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

num_epochs = 10
best_val_accuracy = 0
best_model_state = None
for epoch in range(num_epochs):
    train_loss = train_linear_probe(linear_probe_model, train_loader, criterion, optimizer, epoch)
    val_accuracy = evaluate(linear_probe_model, val_loader)
    print(f'Epoch {epoch}, Train Loss: {train_loss}, Validation Accuracy: {val_accuracy}%')
    

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model_state = linear_probe_model.state_dict()

linear_probe_model.load_state_dict(best_model_state)


test_accuracy = evaluate(linear_probe_model, test_loader)
print(f'Best Validation Accuracy: {best_val_accuracy}%')
print(f'Test Accuracy: {test_accuracy}%')


Error loading model: PytorchStreamReader failed reading zip archive: failed finding central directory
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Epoch 0, Batch 0, Loss: 2.325059175491333
Epoch 0, Batch 10, Loss: 2.0261573791503906
Epoch 0, Batch 20, Loss: 1.869328260421753
Epoch 0, Batch 30, Loss: 1.9303045272827148
Epoch 0, Batch 40, Loss: 1.8603856563568115
Epoch 0, Batch 50, Loss: 1.9464701414108276
Epoch 0, Batch 60, Loss: 1.8822803497314453
Epoch 0, Batch 70, Loss: 1.9040511846542358
Epoch 0, Batch 80, Loss: 1.8852758407592773
Epoch 0, Batch 90, Loss: 1.744970679283142
Epoch 0, Batch 100, Loss: 1.6991333961486816
Epoch 0, Batch 110, Loss: 1.7160401344299316
Epoch 0, Batch 120, Loss: 1.7872779369354248
Epoch 0, Batch 130, Loss: 1.664961338043213
Epoch 0, Batch 140, Loss: 1.7176120281219482
Epoch 0, T

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Subset
from medmnist import PathMNIST
import numpy as np


class SimCLR(nn.Module):
    def __init__(self, base_model, out_dim):
        super(SimCLR, self).__init__()
        self.encoder = self.get_resnet(base_model)
        self.projector = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, out_dim)
        )

    def get_resnet(self, base_model):
        model = models.__dict__[base_model](weights=None)
        model = nn.Sequential(*list(model.children())[:-1])
        return model

    def forward(self, x):
        h = self.encoder(x)
        h = h.squeeze()
        z = self.projector(h)
        return h, z


model = SimCLR(base_model='resnet18', out_dim=128).cuda()


try:
    model.load_state_dict(torch.load('./simclr_pathmnist_pretrained_epoch_25.pth'))
except Exception as e:
    print(f"Error loading model: {e}")

for param in model.encoder.parameters():
    param.requires_grad = False


class LinearProbe(nn.Module):
    def __init__(self, base_encoder):
        super(LinearProbe, self).__init__()
        self.encoder = base_encoder
        self.classifier = nn.Linear(512, 9)  

    def forward(self, x):
        with torch.no_grad():
            h = self.encoder(x)
        h = h.squeeze()
        logits = self.classifier(h)
        return logits


linear_probe_model = LinearProbe(model.encoder).cuda()

PathMNIST_MEAN = [0.73765225, 0.53090023, 0.70307171]
PathMNIST_STD = [0.12319908, 0.17607205, 0.12394462]


data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=PathMNIST_MEAN, std=PathMNIST_STD)
])

train_dataset = PathMNIST(split='train', transform=data_transform, download=True, size=64)
val_dataset = PathMNIST(split='val', transform=data_transform, download=True, size=64)
test_dataset = PathMNIST(split='test', transform=data_transform, download=True, size=64)

num_samples = len(train_dataset)
subset_indices = list(range(num_samples))
np.random.shuffle(subset_indices)
subset_indices = subset_indices[:int(1 * num_samples)]

train_subset = Subset(train_dataset, subset_indices)
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(linear_probe_model.classifier.parameters(), lr=1e-3, weight_decay=1e-6)

def train_linear_probe(model, train_loader, criterion, optimizer, epoch):
    model.train()
    total_loss = 0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.cuda(), labels.squeeze().long().cuda() 
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        if batch_idx % 10 == 0:
            print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}')
    return total_loss / len(train_loader)

def evaluate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.cuda(), labels.squeeze().long().cuda()  # 确保 labels 是 1D 张量
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

num_epochs = 10
best_val_accuracy = 0
best_model_state = None
for epoch in range(num_epochs):
    train_loss = train_linear_probe(linear_probe_model, train_loader, criterion, optimizer, epoch)
    val_accuracy = evaluate(linear_probe_model, val_loader)
    print(f'Epoch {epoch}, Train Loss: {train_loss}, Validation Accuracy: {val_accuracy}%')
    

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model_state = linear_probe_model.state_dict()

linear_probe_model.load_state_dict(best_model_state)

test_accuracy = evaluate(linear_probe_model, test_loader)
print(f'Best Validation Accuracy: {best_val_accuracy}%')
print(f'Test Accuracy: {test_accuracy}%')


Error loading model: PytorchStreamReader failed reading zip archive: failed finding central directory
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Epoch 0, Batch 0, Loss: 2.4099528789520264
Epoch 0, Batch 10, Loss: 2.114763021469116
Epoch 0, Batch 20, Loss: 2.169755697250366
Epoch 0, Batch 30, Loss: 2.00824236869812
Epoch 0, Batch 40, Loss: 1.9558193683624268
Epoch 0, Batch 50, Loss: 1.9336069822311401
Epoch 0, Batch 60, Loss: 1.9814732074737549
Epoch 0, Batch 70, Loss: 1.798601508140564
Epoch 0, Batch 80, Loss: 1.7699127197265625
Epoch 0, Batch 90, Loss: 1.7611044645309448
Epoch 0, Batch 100, Loss: 1.7327957153320312
Epoch 0, Batch 110, Loss: 1.7308229207992554
Epoch 0, Batch 120, Loss: 1.6507103443145752
Epoch 0, Batch 130, Loss: 1.6986031532287598
Epoch 0, Batch 140, Loss: 1.60010826587677
Epoch 0, Batc

# 50 epoch

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Subset
from medmnist import PathMNIST
import numpy as np

class SimCLR(nn.Module):
    def __init__(self, base_model, out_dim):
        super(SimCLR, self).__init__()
        self.encoder = self.get_resnet(base_model)
        self.projector = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, out_dim)
        )

    def get_resnet(self, base_model):
        model = models.__dict__[base_model](weights=None)
        model = nn.Sequential(*list(model.children())[:-1])
        return model

    def forward(self, x):
        h = self.encoder(x)
        h = h.squeeze()
        z = self.projector(h)
        return h, z


model = SimCLR(base_model='resnet18', out_dim=128).cuda()


try:
    model.load_state_dict(torch.load('./simclr_pathmnist_pretrained_epoch_50.pth'))
except Exception as e:
    print(f"Error loading model: {e}")


for param in model.encoder.parameters():
    param.requires_grad = False


class LinearProbe(nn.Module):
    def __init__(self, base_encoder):
        super(LinearProbe, self).__init__()
        self.encoder = base_encoder
        self.classifier = nn.Linear(512, 9)  

    def forward(self, x):
        with torch.no_grad():
            h = self.encoder(x)
        h = h.squeeze()
        logits = self.classifier(h)
        return logits


linear_probe_model = LinearProbe(model.encoder).cuda()

PathMNIST_MEAN = [0.73765225, 0.53090023, 0.70307171]
PathMNIST_STD = [0.12319908, 0.17607205, 0.12394462]


data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=PathMNIST_MEAN, std=PathMNIST_STD)
])

train_dataset = PathMNIST(split='train', transform=data_transform, download=True, size=64)
val_dataset = PathMNIST(split='val', transform=data_transform, download=True, size=64)
test_dataset = PathMNIST(split='test', transform=data_transform, download=True, size=64)

num_samples = len(train_dataset)
subset_indices = list(range(num_samples))
np.random.shuffle(subset_indices)
subset_indices = subset_indices[:int(0.01 * num_samples)]

train_subset = Subset(train_dataset, subset_indices)
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(linear_probe_model.classifier.parameters(), lr=1e-3, weight_decay=1e-6)

def train_linear_probe(model, train_loader, criterion, optimizer, epoch):
    model.train()
    total_loss = 0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.cuda(), labels.squeeze().long().cuda()  # make sure labels is 1D tensor
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        if batch_idx % 10 == 0:
            print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}')
    return total_loss / len(train_loader)

def evaluate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.cuda(), labels.squeeze().long().cuda() 
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

num_epochs = 10
best_val_accuracy = 0
best_model_state = None
for epoch in range(num_epochs):
    train_loss = train_linear_probe(linear_probe_model, train_loader, criterion, optimizer, epoch)
    val_accuracy = evaluate(linear_probe_model, val_loader)
    print(f'Epoch {epoch}, Train Loss: {train_loss}, Validation Accuracy: {val_accuracy}%')
    

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model_state = linear_probe_model.state_dict()

linear_probe_model.load_state_dict(best_model_state)


test_accuracy = evaluate(linear_probe_model, test_loader)
print(f'Best Validation Accuracy: {best_val_accuracy}%')
print(f'Test Accuracy: {test_accuracy}%')


Error loading model: PytorchStreamReader failed reading zip archive: failed finding central directory
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Epoch 0, Batch 0, Loss: 2.2523465156555176
Epoch 0, Batch 10, Loss: 2.102701425552368
Epoch 0, Train Loss: 2.125023873647054, Validation Accuracy: 18.422630947620952%
Epoch 1, Batch 0, Loss: 2.0268633365631104
Epoch 1, Batch 10, Loss: 1.8630499839782715
Epoch 1, Train Loss: 1.9863341490427653, Validation Accuracy: 24.840063974410235%
Epoch 2, Batch 0, Loss: 1.8402773141860962
Epoch 2, Batch 10, Loss: 1.7296574115753174
Epoch 2, Train Loss: 1.902810033162435, Validation Accuracy: 31.827269092363053%
Epoch 3, Batch 0, Loss: 1.8165563344955444
Epoch 3, Batch 10, Loss: 1.8288967609405518
Epoch 3, Train Loss: 1.806272013982137, Validation Accuracy: 32.62694922031188%

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Subset
from medmnist import PathMNIST
import numpy as np


class SimCLR(nn.Module):
    def __init__(self, base_model, out_dim):
        super(SimCLR, self).__init__()
        self.encoder = self.get_resnet(base_model)
        self.projector = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, out_dim)
        )

    def get_resnet(self, base_model):
        model = models.__dict__[base_model](weights=None)
        model = nn.Sequential(*list(model.children())[:-1])
        return model

    def forward(self, x):
        h = self.encoder(x)
        h = h.squeeze()
        z = self.projector(h)
        return h, z


model = SimCLR(base_model='resnet18', out_dim=128).cuda()

try:
    model.load_state_dict(torch.load('./simclr_pathmnist_pretrained_epoch_50.pth'))
except Exception as e:
    print(f"Error loading model: {e}")


for param in model.encoder.parameters():
    param.requires_grad = False


class LinearProbe(nn.Module):
    def __init__(self, base_encoder):
        super(LinearProbe, self).__init__()
        self.encoder = base_encoder
        self.classifier = nn.Linear(512, 9)  

    def forward(self, x):
        with torch.no_grad():
            h = self.encoder(x)
        h = h.squeeze()
        logits = self.classifier(h)
        return logits

linear_probe_model = LinearProbe(model.encoder).cuda()

PathMNIST_MEAN = [0.73765225, 0.53090023, 0.70307171]
PathMNIST_STD = [0.12319908, 0.17607205, 0.12394462]

data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=PathMNIST_MEAN, std=PathMNIST_STD)
])


train_dataset = PathMNIST(split='train', transform=data_transform, download=True, size=64)
val_dataset = PathMNIST(split='val', transform=data_transform, download=True, size=64)
test_dataset = PathMNIST(split='test', transform=data_transform, download=True, size=64)

num_samples = len(train_dataset)
subset_indices = list(range(num_samples))
np.random.shuffle(subset_indices)
subset_indices = subset_indices[:int(0.1 * num_samples)]

train_subset = Subset(train_dataset, subset_indices)
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(linear_probe_model.classifier.parameters(), lr=1e-3, weight_decay=1e-6)

def train_linear_probe(model, train_loader, criterion, optimizer, epoch):
    model.train()
    total_loss = 0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.cuda(), labels.squeeze().long().cuda()  
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        if batch_idx % 10 == 0:
            print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}')
    return total_loss / len(train_loader)

def evaluate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.cuda(), labels.squeeze().long().cuda()  
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

num_epochs = 10
best_val_accuracy = 0
best_model_state = None
for epoch in range(num_epochs):
    train_loss = train_linear_probe(linear_probe_model, train_loader, criterion, optimizer, epoch)
    val_accuracy = evaluate(linear_probe_model, val_loader)
    print(f'Epoch {epoch}, Train Loss: {train_loss}, Validation Accuracy: {val_accuracy}%')
    
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model_state = linear_probe_model.state_dict()
linear_probe_model.load_state_dict(best_model_state)

test_accuracy = evaluate(linear_probe_model, test_loader)
print(f'Best Validation Accuracy: {best_val_accuracy}%')
print(f'Test Accuracy: {test_accuracy}%')


Error loading model: PytorchStreamReader failed reading zip archive: failed finding central directory
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Epoch 0, Batch 0, Loss: 2.3063788414001465
Epoch 0, Batch 10, Loss: 2.1800320148468018
Epoch 0, Batch 20, Loss: 1.9225044250488281
Epoch 0, Batch 30, Loss: 1.964726448059082
Epoch 0, Batch 40, Loss: 1.9487274885177612
Epoch 0, Batch 50, Loss: 1.8286046981811523
Epoch 0, Batch 60, Loss: 1.811076283454895
Epoch 0, Batch 70, Loss: 1.6507103443145752
Epoch 0, Batch 80, Loss: 1.8722964525222778
Epoch 0, Batch 90, Loss: 1.7462856769561768
Epoch 0, Batch 100, Loss: 1.7835569381713867
Epoch 0, Batch 110, Loss: 1.8792753219604492
Epoch 0, Batch 120, Loss: 1.6946680545806885
Epoch 0, Batch 130, Loss: 1.7640433311462402
Epoch 0, Batch 140, Loss: 1.6019096374511719
Epoch 0,

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Subset
from medmnist import PathMNIST
import numpy as np

class SimCLR(nn.Module):
    def __init__(self, base_model, out_dim):
        super(SimCLR, self).__init__()
        self.encoder = self.get_resnet(base_model)
        self.projector = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, out_dim)
        )

    def get_resnet(self, base_model):
        model = models.__dict__[base_model](weights=None)
        model = nn.Sequential(*list(model.children())[:-1])
        return model

    def forward(self, x):
        h = self.encoder(x)
        h = h.squeeze()
        z = self.projector(h)
        return h, z

model = SimCLR(base_model='resnet18', out_dim=128).cuda()

try:
    model.load_state_dict(torch.load('./simclr_pathmnist_pretrained_epoch_50.pth'))
except Exception as e:
    print(f"Error loading model: {e}")


for param in model.encoder.parameters():
    param.requires_grad = False


class LinearProbe(nn.Module):
    def __init__(self, base_encoder):
        super(LinearProbe, self).__init__()
        self.encoder = base_encoder
        self.classifier = nn.Linear(512, 9)  
        
    def forward(self, x):
        with torch.no_grad():
            h = self.encoder(x)
        h = h.squeeze()
        logits = self.classifier(h)
        return logits

linear_probe_model = LinearProbe(model.encoder).cuda()

PathMNIST_MEAN = [0.73765225, 0.53090023, 0.70307171]
PathMNIST_STD = [0.12319908, 0.17607205, 0.12394462]


data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=PathMNIST_MEAN, std=PathMNIST_STD)
])


train_dataset = PathMNIST(split='train', transform=data_transform, download=True, size=64)
val_dataset = PathMNIST(split='val', transform=data_transform, download=True, size=64)
test_dataset = PathMNIST(split='test', transform=data_transform, download=True, size=64)


num_samples = len(train_dataset)
subset_indices = list(range(num_samples))
np.random.shuffle(subset_indices)
subset_indices = subset_indices[:int(1 * num_samples)]

train_subset = Subset(train_dataset, subset_indices)
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(linear_probe_model.classifier.parameters(), lr=1e-3, weight_decay=1e-6)

def train_linear_probe(model, train_loader, criterion, optimizer, epoch):
    model.train()
    total_loss = 0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.cuda(), labels.squeeze().long().cuda()  
                
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        if batch_idx % 10 == 0:
            print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}')
    return total_loss / len(train_loader)

def evaluate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.cuda(), labels.squeeze().long().cuda()  
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

num_epochs = 10
best_val_accuracy = 0
best_model_state = None
for epoch in range(num_epochs):
    train_loss = train_linear_probe(linear_probe_model, train_loader, criterion, optimizer, epoch)
    val_accuracy = evaluate(linear_probe_model, val_loader)
    print(f'Epoch {epoch}, Train Loss: {train_loss}, Validation Accuracy: {val_accuracy}%')
    

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model_state = linear_probe_model.state_dict()

linear_probe_model.load_state_dict(best_model_state)

test_accuracy = evaluate(linear_probe_model, test_loader)
print(f'Best Validation Accuracy: {best_val_accuracy}%')
print(f'Test Accuracy: {test_accuracy}%')


Error loading model: PytorchStreamReader failed reading zip archive: failed finding central directory
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Using downloaded and verified file: C:\Users\xie\.medmnist\pathmnist_64.npz
Epoch 0, Batch 0, Loss: 2.3683950901031494
Epoch 0, Batch 10, Loss: 2.126798152923584
Epoch 0, Batch 20, Loss: 2.0432913303375244
Epoch 0, Batch 30, Loss: 1.920710802078247
Epoch 0, Batch 40, Loss: 1.8767074346542358
Epoch 0, Batch 50, Loss: 1.9641681909561157
Epoch 0, Batch 60, Loss: 1.799139380455017
Epoch 0, Batch 70, Loss: 1.8516727685928345
Epoch 0, Batch 80, Loss: 1.8467769622802734
Epoch 0, Batch 90, Loss: 1.7184334993362427
Epoch 0, Batch 100, Loss: 1.8639758825302124
Epoch 0, Batch 110, Loss: 1.7580262422561646
Epoch 0, Batch 120, Loss: 1.8155076503753662
Epoch 0, Batch 130, Loss: 1.8071348667144775
Epoch 0, Batch 140, Loss: 1.7139227390289307
Epoch 0, 